In [28]:
import requests
import re
from bs4 import BeautifulSoup

cardrush_url = "https://www.cardrush-pokemon.jp/phone/new"
url = f"{cardrush_url}/new?page=1"

def extract_goods_and_prices(soup):
    results = {}

    item_data_divs = soup.find_all('div', class_='item_data')

    for div in item_data_divs:
        goods_name = div.find('span', class_='goods_name')
        selling_price = div.find('p', class_='selling_price')
        model_number = div.find('span', class_ ='model_number')

        if goods_name and selling_price:
            #Extracting name of the card
            goods_name_text = goods_name.text.strip()
            #Extracting ID of the card
            match = re.search(r'\{(\d+/\d+)\}', goods_name_text)
            if match:
                goods_id = match.group(1)
                
                #Extracting the price of the card
                selling_price_text = selling_price.text.strip()
                price = int(re.sub(r'\D+', '', selling_price_text))
                
                #Extracting the model of the card
                model_number_text = model_number.text.strip()
                model_value = re.search(r'S([^]]+)\]', model_number_text)

                if model_value:
                    model_number = 'S' + model_value.group(1)
                    result = {
                        'unique_id': model_number + '-' + goods_id,
                        'JP_name': goods_name_text,
                        'JP_price': price,
                        'US_price': -1,
                        'US_name': ''
                    }
                    results[result['unique_id']] = result

    return results

def fetch_and_parse(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to fetch the page, status code: {response.status_code}")
        return None

print("Reading Japanese Pokemon Card Info...")
# Fetch the first page
soup = fetch_and_parse(url)

if soup:
    # Extract goods_name and selling_price from the first page
    all_results = extract_goods_and_prices(soup)
    # Find the last page number
    pagebottom_div = soup.find('div', id='pagerbottom')

    if pagebottom_div:
        last_page_link = pagebottom_div.find_all('a', class_='pager_btn')[-2]
        last_page_number = int(last_page_link['href'].split('=')[-1])
        
        
        # Loop through the remaining pages
        for page_number in range(2, last_page_number + 1):
            print(page_number)
            page_url = f"{base_url}/new?page={page_number}"
            soup = fetch_and_parse(page_url)

            if soup:
                # Extract goods_name and selling_price from the current page
                results = extract_goods_and_prices(soup)
                all_results.update(results)
                
    print(len(all_results))
    # Print the extracted data
    for result in all_results.values():
        print(f"Unique ID: {result['unique_id']}")
        print(f"JP Name: {result['JP_name']}")
        print(f"JP Price: {result['JP_price']}")
        print(f"US Name: {result['US_name']}")
        print(f"US Price: {result['US_price']}")
        
        print("\n")
    

print("Finished reading Japanese Pokemon Card Info")


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
1452
Unique ID: S10b-055/071
JP Name: 〔状態A-〕かがやくイーブイ【K】{055/071}
JP Price: 830
US Name: 
US Price: -1


Unique ID: SV2D-067/071
JP Name: 勇気のおまもり【U】{067/071}
JP Price: 220
US Name: 
US Price: -1


Unique ID: SV2P-066/071
JP Name: ☆SALE☆すごいつりざお【U】{066/071}
JP Price: 180
US Name: 
US Price: -1


Unique ID: SV2P-071/071
JP Name: ☆SALE☆リバーサルエネルギー【U】{071/071}
JP Price: 120
US Name: 
US Price: -1


Unique ID: SV2D-069/071
JP Name: ナンジャモ【U】{069/071}
JP Price: 580
US Name: 
US Price: -1


Unique ID: SV2D-097/071
JP Name: ディンルーex【UR】{097/071}
JP Price: 1580
US Name: 
US Price: -1


Unique ID: SV2P-097/071
JP Name: パオジアンex【UR】{097/071}
JP Price: 2780
US Name: 
US Price: -1


Unique ID: SV2D-098/071
JP Name: スーパーエネルギー回収【UR】{098/071}
JP Price: 2580
US Name: 
US Price: -1


Unique ID: SV2P-098/071
JP Name: すごいつりざお【UR】{098/071}
JP Price: 3280
US Name: 
US Price: -1


Unique ID: SV2D-099/071
JP Name: 基本草エネルギー(SV

US Price: -1


Unique ID: SVAM-011/023
JP Name: グルトン【-】{011/023}
JP Price: 80
US Name: 
US Price: -1


Unique ID: SVAM-012/023
JP Name: エネルギーつけかえ【-】{012/023}
JP Price: 120
US Name: 
US Price: -1


Unique ID: SVAM-013/023
JP Name: エネルギー転送【-】{013/023}
JP Price: 80
US Name: 
US Price: -1


Unique ID: SVAM-014/023
JP Name: きずぐすり【-】{014/023}
JP Price: 80
US Name: 
US Price: -1


Unique ID: SVAM-015/023
JP Name: ネストボール【-】{015/023}
JP Price: 420
US Name: 
US Price: -1


Unique ID: SVAM-016/023
JP Name: ハイパーボール【-】{016/023}
JP Price: 80
US Name: 
US Price: -1


Unique ID: SVAM-017/023
JP Name: ポケギア3.0【-】{017/023}
JP Price: 80
US Name: 
US Price: -1


Unique ID: SVAM-018/023
JP Name: ポケモンいれかえ【-】{018/023}
JP Price: 50
US Name: 
US Price: -1


Unique ID: SVAM-019/023
JP Name: ポケモンキャッチャー【-】{019/023}
JP Price: 50
US Name: 
US Price: -1


Unique ID: SVAM-020/023
JP Name: ジャッジマン(SVAM)【-】{020/023}
JP Price: 80
US Name: 
US Price: -1


Unique ID: SVAM-021/023
JP Name: たんぱんこぞう(SVAM)【-】{021/023}
JP Price:

JP Price: 780
US Name: 
US Price: -1


Unique ID: S10b-091/071
JP Name: 〔状態A-〕ミュウツーVSTAR【UR】{091/071}
JP Price: 3080
US Name: 
US Price: -1


Unique ID: S10b-073/071
JP Name: 〔状態C〕ミュウツーV【SR】{073/071}
JP Price: 480
US Name: 
US Price: -1


Unique ID: S10b-071/071
JP Name: 〔状態C〕ポケストップ【U】{071/071}
JP Price: 50
US Name: 
US Price: -1


Unique ID: S10b-067/071
JP Name: 〔状態B〕ルアーモジュール【U】{067/071}
JP Price: 30
US Name: 
US Price: -1


Unique ID: S10b-066/071
JP Name: 〔状態B〕ふしぎなアメ【U】{066/071}
JP Price: 50
US Name: 
US Price: -1


Unique ID: S10b-065/071
JP Name: 〔状態B〕ふかそうち【U】{065/071}
JP Price: 30
US Name: 
US Price: -1


Unique ID: S10b-059/071
JP Name: 〔状態B〕ケッキングV【RR】{059/071}
JP Price: 50
US Name: 
US Price: -1


Unique ID: S10b-056/071
JP Name: 〔状態A-〕カビゴン【R】{056/071}
JP Price: 70
US Name: 
US Price: -1


Unique ID: S10b-053/071
JP Name: 〔状態B〕メタモン(シール付/R仕様)【-】{053/071}
JP Price: 80
US Name: 
US Price: -1


Unique ID: S10b-050/071
JP Name: 〔状態B〕カイリューVSTAR【RRR】{050/071}
JP Price: 50
US Name: 
U


Unique ID: S8a-P-011/025
JP Name: マグマ団のグラードン(25th)【P】{011/025}
JP Price: 580
US Name: 
US Price: -1


Unique ID: S8a-P-012/025
JP Name: ブラッキー☆(25th)【P】{012/025}
JP Price: 14800
US Name: 
US Price: -1


Unique ID: S8a-P-013/025
JP Name: ロケット団の幹部(25th)【P】{013/025}
JP Price: 480
US Name: 
US Price: -1


Unique ID: S8a-P-014/025
JP Name: ミュウex(25th)【P】{014/025}
JP Price: 2380
US Name: 
US Price: -1


Unique ID: S8a-P-015/025
JP Name: サーナイトex δ種(25th)【P】{015/025}
JP Price: 880
US Name: 
US Price: -1


Unique ID: S8a-P-016/025
JP Name: ネンドール(25th)【P】{016/025}
JP Price: 480
US Name: 
US Price: -1


Unique ID: S8a-P-017/025
JP Name: レントラーGL LV.X(25th)【P】{017/025}
JP Price: 380
US Name: 
US Price: -1


Unique ID: S8a-P-018/025
JP Name: ガブリアスC LV.X(25th)【P】{018/025}
JP Price: 780
US Name: 
US Price: -1


Unique ID: S8a-P-019/025
JP Name: ドンファン(25th)【P】{019/025}
JP Price: 380
US Name: 
US Price: -1


Unique ID: S8a-P-020/025
JP Name: レシラム(25th)【P】{020/025}
JP Price: 4480
US Name: 
US Price: -1



In [4]:
import requests

url = "http://127.0.0.1:5000/store_data"
data = {
    "unique_id": "S123-456/789",
    "JP_name": "ポケモンカード",
    "JP_price": 1000,
    "US_name": "Pokemon Card",
    "US_price": 10
}
response = requests.post(url, json=data)
print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)